<a href="https://colab.research.google.com/github/jsansao/teic-20231/blob/main/TEIC_Licao32_Nanogpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📖 Demonstração: Treinando um nanoGPT do Zero

Este notebook irá demonstrar como treinar um modelo de linguagem (GPT) do zero usando o repositório `nanoGPT` de Andrej Karpathy.

Vamos treinar um modelo *baseado em caracteres* no corpus de "Tiny Shakespeare" para que ele aprenda a "falar" como Shakespeare.

### Passo 1: Configurar o Ambiente

Primeiro, precisamos garantir que estamos usando uma GPU.

1.  Vá em **Ambiente de execução -> Alterar o tipo de ambiente de execução**.
2.  Selecione **T4 GPU** (ou qualquer GPU disponível) e clique em `Salvar`.

Agora, vamos verificar se o PyTorch consegue detectar a GPU.

In [1]:
import torch

if torch.cuda.is_available():
    print("GPU (CUDA) detectada! 👍")
    print(f"Nome da GPU: {torch.cuda.get_device_name(0)}")
else:
    print("GPU não detectada. 👎")
    print("Por favor, habilite a GPU no 'Ambiente de execução' para continuar.")

GPU (CUDA) detectada! 👍
Nome da GPU: Tesla T4


### Passo 2: Clonar o nanoGPT e Instalar Dependências

Vamos baixar o código do GitHub e instalar as bibliotecas necessárias.

In [2]:
# 1. Clona o repositório oficial
!git clone https://github.com/karpathy/nanoGPT.git

# 2. Entra no diretório do projeto
# Usamos o comando mágico %cd para mudar o diretório do notebook
%cd nanoGPT

# 3. Instala as bibliotecas necessárias
!pip install torch numpy transformers datasets tiktoken wandb tqdm

Cloning into 'nanoGPT'...
remote: Enumerating objects: 686, done.
remote: Total 686 (delta 0), reused 0 (delta 0), pack-reused 686 (from 1)
Receiving objects: 100% (686/686), 974.06 KiB | 3.78 MiB/s, done.
Resolving deltas: 100% (380/380), done.
/content/nanoGPT


### Passo 3: Obter e Preparar os Dados (Corpus de Shakespeare)

O repositório já vem com um script que baixa o "Tiny Shakespeare" (um arquivo `input.txt`). Este script irá "tokenizar" o texto, ou seja, converter todos os caracteres únicos em números.

In [3]:
# Este script baixa o input.txt e o processa,
# criando os arquivos 'train.bin' e 'val.bin' na pasta 'data/shakespeare_char'
!python data/shakespeare_char/prepare.py

length of dataset in characters: 1,115,394
all the unique characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65
train has 1,003,854 tokens
val has 111,540 tokens


### Passo 4: Treinar o Modelo!

Este é o momento principal. Vamos iniciar o treinamento usando o arquivo de configuração `train_shakespeare_char.py`.

Preste atenção no output: você verá o `iter` (iteração) e o `loss` (erro). O `loss` deve diminuir progressivamente, provando que o modelo está aprendendo.

In [4]:
# Inicia o treinamento!
# Isso pode levar de 5 a 15 minutos para mostrar resultados decentes.
# O modelo salvará 'checkpoints' na pasta 'out-shakespeare-char'
!python train.py config/train_shakespeare_char.py

Overriding config with config/train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 200
log_interval = 10 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 64
block_size = 256 # context of up to 256 previous characters

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 5000
lr_decay_iters = 5000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of 

**Nota sobre o Treinamento:**

O script padrão roda por `max_iters = 5000` iterações. Para uma demonstração, você não precisa esperar até o fim. Você pode **interromper a execução manualmente** (clicando no botão "Stop") quando o `val loss` (loss de validação) estiver baixo (algo em torno de 1.5 ou 1.6). O modelo já será capaz de gerar texto reconhecível.

### Passo 5: Gerar Texto (A Demonstração)

Após o modelo ter treinado por um tempo, ele terá salvo um *checkpoint*. Vamos usar o script `sample.py` para carregar esse checkpoint e gerar texto novo.

In [6]:
# Gera 500 caracteres de texto novo
# Ele automaticamente carrega o checkpoint mais recente da pasta 'out-shakespeare-char'
!python sample.py --out_dir=out-shakespeare-char

Overriding: out_dir = out-shakespeare-char
number of parameters: 10.65M
Loading meta from data/shakespeare_char/meta.pkl...


ANGELO:
And cowards it with the present souls.
Stirrah, we have heard the fuults of your desires,
But were you fall out of the other to the heart
Of whose black is ensented to the time
Give and quietness in warms. Ah, noble Peter!
Thou hast a childishes wonder'd and Romeo,
Is a less present to the laoyWing to thy highness
Without the gentle crown, and he is
To kill thy father and great shall divise on thee
Married with thy household obeys:
Sad all his grace hath couragainted Rutland,
And from t
---------------

Men pardon, marry, I will give me
That I have sent forth and madness; and yet I will sue.

ISABELLA:
I fear, sir, I would to instruct me.

ANGELO:
I am not over the part of most grave to time:
Here comes me that I shall speak again
Whose looks does of the heavens.

ANGELO:
Think'st thou, why I may make the world?

ISABELLA:
Well, good my lord, we have not

**O que observar:**

O texto gerado deve se parecer com o inglês de Shakespeare. Você verá nomes de personagens (ex: `KING:`, `JULIET:`) e palavras no estilo arcaico. Isso demonstra que o modelo não *decorou* o texto, mas sim *aprendeu os padrões e o estilo* do corpus.

---

### (Opcional) Passo 6: Treinar com seu Próprio Corpus

Se quiser treinar com um corpus diferente (ex: poemas em português):

1.  No painel de arquivos à esquerda, navegue até `nanoGPT/data/`.
2.  Crie uma nova pasta (ex: `meu_corpus`).
3.  Faça o upload do seu arquivo de texto para essa pasta e renomeie-o para `input.txt`.
4.  Execute os comandos abaixo.

In [ ]:
# 1. Crie o diretório (apenas para garantir)
!mkdir -p data/meu_corpus

#
# ----------------------------------------------------------------------
# (AGORA, FAÇA O UPLOAD DO SEU 'input.txt' PARA A PASTA 'data/meu_corpus')
# ----------------------------------------------------------------------
#

# 2. Copie o script de preparação
!cp data/shakespeare_char/prepare.py data/meu_corpus/prepare.py

# 3. Rode a preparação (ele irá tokenizar seu 'input.txt')
# (Descomente a linha abaixo após fazer o upload)
#!python data/meu_corpus/prepare.py

In [ ]:
# 4. Copie o arquivo de configuração
!cp config/train_shakespeare_char.py config/train_meu_corpus.py

# 5. EDITE O NOVO ARQUIVO DE CONFIG:
#   - Abra 'config/train_meu_corpus.py' no editor de arquivos
#   - Mude a linha: dataset = 'shakespeare_char' PARA dataset = 'meu_corpus'
#   - Mude a linha: out_dir = 'out-shakespeare-char' PARA out_dir = 'out-meu-corpus'

# 6. TREINE com seu corpus
# (Descomente a linha abaixo após editar o arquivo de config)
#!python train.py config/train_meu_corpus.py

In [ ]:
# 7. GERE TEXTO a partir do seu modelo
# (Descomente a linha abaixo após o treinamento)
#!python sample.py --out_dir=out-meu_corpus --n=500